<a href="https://colab.research.google.com/github/AmeeraBawazir/self-supervised-learning-img-classification/blob/main/SSL_cats_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras import backend as K
dataset_name = 'cats_vs_dogs'
filePath = '/content/gdrive/MyDrive/'
(Train, Test), ds_info= tfds.load(name=dataset_name, split=['train[:80%]', 'train[80%:]'], with_info=True, data_dir=filePath)
X_train=[]
y_train= []
for item in Train.take(-1):
   image=item["image"]
   label=item['label']
   image = tf.image.resize(image, (120, 120))
   image = image / 255.0
   image=image.numpy()
   label=label.numpy()
   #plt.imshow(image)
   #plt.show()
   X_train.append(image)
   y_train.append(label)
X_train=np.array(X_train)
y_train=np.array(y_train) 

X_test=[]
y_test= []
for item in Test.take(-1):
   image=item["image"]
   label=item['label']
   image = tf.image.resize(image, (120, 120 ))
   image = image / 255.0
   image=image.numpy()
   label=label.numpy()
   #plt.imshow(image)
   #plt.show()
   X_test.append(image)
   y_test.append(label)
X_test=np.array(X_test)
y_test=np.array(y_test)

Train=[X_train,y_train]
Test=[X_test,y_test]

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
import numpy as np
import os
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import random
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
def preprocess_data(x_train,y_train,x_test,y_test):
        x_train_mean = np.mean(x_train, axis=0)
        x_train -= x_train_mean
        x_test -= x_train_mean
        return x_train,y_train,x_test,y_test

In [ ]:
def rotate_image(im):
  iterations = random.randint(0,3)
  y = [0,0,0,0]
  for i in range(iterations):
    im = np.rot90(im)
  y[iterations]=1
  return im,y

In [ ]:
def make_rotated_data(Train,Test):
    xy_rot_train = list(zip(*[rotate_image(im) for im in Train[0]]))
    xy_rot_test = list(zip(*[rotate_image(im) for im in Test[0]]))
    x_rot_train=np.array(xy_rot_train[0][:])
    y_rot_train=np.array(xy_rot_train[1][:])
    x_rot_test=np.array(xy_rot_test[0][:])
    y_rot_test=np.array(xy_rot_test[1][:])
    x_rot_train_mean = np.mean(x_rot_train, axis=0)
    x_rot_train -= x_rot_train_mean
    x_rot_test -= x_rot_train_mean
    return x_rot_train,y_rot_train,x_rot_test,y_rot_test

In [ ]:
## Resnet Implementation taken from David Yang
lr = 1e-3
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalizatio:
           for res_block in range(num_rn):
              x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def resnet_v1(input_shape, depth, num_classes=10): 
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)
    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2
    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=7)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)
    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def make_ssl_backbone(Train,Test,save_dir,input_shape=(120,120,3),n=3,model_name='cats_dogs.h5'):

    x_rot_train,y_rot_train,x_rot_test,y_rot_test = make_rotated_data(Train,Test)
    
    
    # Computed depth from supplied model parameter n
    depth = n * 6 + 2
    
    resnet_model= resnet_v1(input_shape=input_shape, depth=depth)
    x = Dense(4,activation='softmax')(resnet_model.layers[-2].output)
    model = keras.Model(resnet_model.inputs,x)

    filepath = os.path.join(save_dir, model_name)
    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath=filepath,
                                 monitor='val_accuracy',
                                 verbose=1,
                                 save_best_only=True)
    lr_scheduler = LearningRateScheduler(lr_schedule)
    lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=10,
                                   min_lr=0.5e-6)
    callbacks = [checkpoint, lr_reducer, lr_scheduler]

    optimizer = keras.optimizers.Adam()

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    history_SSL = model.fit(x_rot_train,y_rot_train,validation_data=(x_rot_test,y_rot_test),epochs=15,callbacks=callbacks)
    
    history_filepath = save_dir
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history_SSL.history) 
    hist_csv_file = '/content/gdrive/MyDrive/saving/cats_dogs/cats_dogs_ssl.csv'
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)
    #model.save(save_dir)

In [ ]:
make_ssl_backbone(Train,Test,save_dir='/content/gdrive/MyDrive/saving/cats_dogs')

In [ ]:
def train_models_datasplit(X_train,y_train,X_test,y_test,
                               batch_size = 32,epochs=12,data_augmentation=True,
                               num_classes=2,
                               n = 3,ssl_path=None,dataset='cats_dogs',input_shape=(120,120,3),
                           data_fraction_increment=0.1,num_increments=10,random_state=689):
     
    ###Calcualte number of many kfolds splits 
    n_splits=int(1/data_fraction_increment)
        
        
    # Convert class vectors to binary class matrices.
    y_test = keras.utils.to_categorical(y_test, num_classes)
    # Computed depth from supplied model parameter n
    depth = n * 6 + 2
    # Model name, depth and version
    model_type = 'ResNet%d' % depth
    
    ###Calcualte number of many kfolds splits 
    n_splits=int(1/data_fraction_increment)
    
    ##Ensure that data increments consist of class balanced data
    skf = StratifiedKFold(n_splits=n_splits,random_state=random_state)
    
    histories = []
    for i,index  in enumerate(skf.split(X_train,y=y_train)):
        if i>num_increments-1:
            break
        if i > 0:
            test_index = np.concatenate((test_index,index[1]))
        else:

          332
            test_index=index[1]
        y_train_t = keras.utils.to_categorical(y_train, num_classes)[test_index]
        if ssl_path:
            ssl_model=keras.models.load_model(ssl_path)
            x=model=ssl_model.layers[-2].output
            x = Dense(2,activation='softmax')(x)
            model = keras.Model(ssl_model.inputs,x)
        else:
            model = resnet_v1(input_shape=input_shape, depth=depth,num_classes=2)
        
        model.compile(loss='binary_crossentropy',
                      optimizer=Adam(learning_rate=lr_schedule(0)),
                      metrics=['accuracy'])
        # Prepare model model saving directory.
        save_dir = os.path.join('/content/gdrive/MyDrive/saving/cats_dogs')
        model_name = dataset +"_"+ str((i/n_splits)*100)+".h5"
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
        filepath = os.path.join(save_dir, model_name)
        #filepath=os.path.join(filepath,".h5")
        
        # Prepare callbacks for model saving and for learning rate adjustment.
        checkpoint = ModelCheckpoint(filepath=filepath,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_best_only=True)
        lr_scheduler = LearningRateScheduler(lr_schedule)
        lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                       cooldown=0,
                                       patience=5,
                                       min_lr=0.5e-6)
        callbacks = [checkpoint, lr_reducer, lr_scheduler]
        
        # Run training, with or without data augmentation.
        if not data_augmentation:
            model.fit(X_train, y_train,
                      batch_size=batch_size,
                      epochs=epochs,
                      validation_data=(x_test, y_test),
                      shuffle=True,
                      callbacks=callbacks)
        else:
            # This will do preprocessing and realtime data augmentation:
            
            histories.append(model.fit(X_train[test_index], y_train_t,batch_size=batch_size,
                                epochs=epochs, verbose=1,validation_data=(X_test,y_test),
                                callbacks=callbacks))
        # Record performance of trained model.
        train_maxes = []
        val_maxes = []
        val_loss = []
        train_loss = []
        if ssl_path:
            record_path = '/content/gdrive/MyDrive/saving/cats_dogs/' +dataset+'_self_supervised.csv'
        else:
            record_path = '/content/gdrive/MyDrive/saving/cats_dogs/' +dataset+'_supervised.csv'

        for i in range(len(histories)):
            val_maxes.append(max(histories[i].history['val_accuracy']))
            train_maxes.append(max(histories[i].history['accuracy']))
            val_loss.append(min(histories[i].history['val_loss']))
            train_loss.append(min(histories[i].history['loss']))
        pd.DataFrame({'Val_Accuracy':val_maxes,
                      'Train_Accuracy':train_maxes,
                      'Val_Loss':val_loss,'Train_Loss':train_loss}).to_csv(record_path)

In [ ]:
### Run experiment WITHOUT self-supervision
x_train, y_train, x_test, y_test = preprocess_data(X_train, y_train, X_test, y_test)
train_models_datasplit(x_train, y_train, x_test, y_test,batch_size=32,num_increments=10,dataset='cats_dogs')

In [ ]:
### Run experiment with WITH self-supervision
x_train, y_train, x_test, y_test = preprocess_data(x_train, y_train, x_test, y_test)
train_models_datasplit(x_train, y_train, x_test, y_test,batch_size=32,num_increments=10,dataset='cats_dogs_ssl',ssl_path='/content/gdrive/MyDrive/saving/cats_dogs/cats_dogs.h5')

In [ ]:
#Visualization for downstream Accuracy and Loss 
df_sl = pd.read_csv('cats_vs_dogs_supervised.csv')
df_ssl = pd.read_csv('cats_vs_dogs_self_supervised.csv')
df_sl['Model_Type']='Supervised'
df = pd.concat((df_sl,df_ssl))
df['Model_Type'][len(df_ssl):]='Self_Supervised'
df['Number_of_labeled_data'] = (df['Unnamed: 0']+1) *18610*0.1 #18610 is size of train set, 0.1 is data_fraction_increment


figure = plt.figure(figsize=(15,7))
g = sns.lineplot(y='Val_Accuracy',x='Number_of_labeled_data',hue='Model_Type',data=df)
g.set(xticks=df['Number_of_labeled_data'].values)
g.set_ylim(0.7,1)
title='Accuracy Self-Supervised vs Only Supervised'
g.set_title(title, fontsize=25)
g.set_xlabel("Number of training data",fontsize=25) #x label + font size 
g.set_ylabel("Validation accuracy",fontsize=25)#y label + font size 
g.tick_params(labelsize=25)#size of num in x and y axis 


figure = plt.figure(figsize=(15,7))
f = sns.lineplot(y='Val_Loss',x='Number_of_labeled_data',hue='Model_Type',data=df)
f.set(xticks=df['Number_of_labeled_data'].values)
title='Loss Self-Supervised vs Only Supervised'
f.set_title(title, fontsize=25)
f.set_xlabel("Number of training data",fontsize=25)
f.set_ylabel("Validation loss",fontsize=25)
f.set_ylim(0,1)
f.tick_params(labelsize=25)
plt.show()

In [ ]:
##visualize ssl model  filters by loading the trained model 
import matplotlib
from matplotlib import pyplot

save_dir = os.getcwd()
model_name='Restnetv1_SL_.h5' 
filepath = os.path.join(save_dir, model_name)

#trying to load the saved model in oredr to visualize the filter 
model=keras.models.load_model(filepath)
model.summary() 
tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
## summarize filter shapes
for layer in model.layers:
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    #get filter weights
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape)
    
#retrieve weights from the second hidden layer
filters, biases = model.layers[1].get_weights()
filters.shape #in first layer: 16 blocks of 3 matrix each is 3x3 (3,3,3,16)



# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
filters

# plot first few filters
n_filters, ix = 16, 1 #determins how many filters to plot
for i in range(n_filters):
    # get the filter
    f = filters[:, :, :, i]
    # plot each channel separately
    for j in range(3): 
        # specify subplot and turn of axis
        ax = pyplot.subplot(n_filters, 3, ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        pyplot.imshow(f[:, :, j], cmap='gray')
        ix += 1
# show the figure
pyplot.show()

In [ ]:
#visualize feature maps
save_dir = os.getcwd()
model_name='Restnetv1_SL_.h5'
filepath = os.path.join(save_dir, model_name)
model=keras.models.load_model(filepath)
# summarize feature map shapes
for i in range(len(model.layers)):
	layer = model.layers[i]
	# check for convolutional layer
	if 'conv' not in layer.name:
		continue
	# summarize output shape
	print(i, layer.name, layer.output.shape)
# redefine model to output right after the first hidden layer
model = Model(inputs=model.inputs, outputs=model.layers[1].output)
#select one image from train data 
img = Train[0][0]
# expand dimensions so that it represents a single 'sample'
img = np.expand_dims(img, axis=0)
feature_maps = model.predict(img)


# plot all 16 maps in an 4x4 squares
square = 4
ix = 1
for _ in range(square):
	for _ in range(square):
		# specify subplot and turn of axis
		ax = pyplot.subplot(square, square, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot filter channel in grayscale
		pyplot.imshow(feature_maps[0, :, :, ix-1], cmap='gray')
		ix += 1
# show the figure
pyplot.show()